In [12]:
!pip install datasets tqdm pandas

In [13]:
!pip install sentencepiece

In [14]:
!pip install transformers

In [15]:
!pip install wandb

In [16]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [17]:
!nvidia-smi

Mon Jan 22 08:12:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
!pip install nltk
!pip install --upgrade pip

In [19]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
import random
import numpy as np
import torch
import datasets
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [21]:
pd.set_option('display.max_colwidth', None)

In [22]:
df = pd.read_csv('/kaggle/input/dataset-corr/train8020.csv')
df.shape


(20084, 10)

In [23]:
from transformers import (
    T5ForConditionalGeneration, AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader


In [24]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)


In [25]:
train_df = pd.read_csv('/kaggle/input/dataset-corr/train8020.csv')
test_df = pd.read_csv('/kaggle/input/dataset-corr/test8020.csv')

In [26]:
model_name = 'csebuetnlp/banglat5'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [27]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-v0m0i8ma
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-v0m0i8ma
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6857 sha256=751ca54cb2f5c5622e8b023cc97acfc1a61560baa336a83fd8cb0515e51a8f94
  Stored in directory: /tmp/pip-ephem-wheel-cache-n53_a0dw/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=7d1842a96fd9347be081f059153a81

In [28]:
from normalizer import normalize
train_df['Comment'] = train_df['Comment'].apply(normalize)
train_df['Correct Form'] = train_df['Correct Form'].apply(normalize)
test_df['Comment'] = test_df['Comment'].apply(normalize)
test_df['Correct Form'] = test_df['Correct Form'].apply(normalize)

In [29]:
train_df.shape, test_df.shape

((20084, 10), (5022, 10))

In [30]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

train_df['input_token_len'] = train_df['Comment'].apply(calc_token_len)
test_df['input_token_len'] = test_df['Comment'].apply(calc_token_len)
test_df.head()

,Video ID,Channel name,Time of Publishing,Title,Genre,Comment,Error,Category,Correct Form,Unnamed: 9,input_token_len
0,62d6PfZxky0,Bangla Bhuter Golpo,2023-01-16T13:00:48Z,Bhuter Cartoon - Railway Station at 2am Night (True Story) Train Horror Story | Bangla Bhuter Golpo,Entertainment,কথা হচ্ছে ওরা এতো ট্রেন পেল কোথায় 🙄 যে এক ঘন্টা পর পর ট্রেন আসতেছে 😳,1,Multiple Errors,কথা হচ্ছে ওরা এত ট্রেন পেল কোথায় 🙄 যে এক ঘণ্টা পর পর ট্রেন আসছে 😳,NaN,20
1,xgMi3vUaw5Y,Baseera,2022-04-02T19:03:14Z,সিরাহ ৭ – অ্যাবিসিনিয়া | Bangla Seerah,Miscellaneous,"প্রিয় ভাই আমার,, আপনি আর বেশি বেশি ভিডিও ছাড়েন,, যেন সুনে আমরা আরো বেশি উপকৃর্ত হতে পারি,,, অনেক অপেক্ষায় থাকি।। আল্লাহ জন্য ভালোবাসি আপনাকে।।",1,Spelling,"প্রিয় ভাই আমার,, আপনি আর বেশি বেশি ভিডিও ছাড়েন,, যেন শুনে আমরা আরো বেশি উপকৃত হতে পারি,,, অনেক অপেক্ষায় থাকি।। আল্লাহ জন্য ভালোবাসি আপনাকে।।",NaN,37
2,8Q7mPmy9eTI,BANGLADESHI REACTOR,2023-01-22T07:00:02Z,লাইভে খেজুর পাতার জা*ইঙ্গা বিক্রি করছে টনি আপা | Salman Muqtadir female version | দাদি দাদা কই,Entertainment,"আপু তোমাকে ধন্যবাদ এইভাবে সবাইকে রিপ্লাই দিয়ে শিক্ষা দেওয়ার জন্য,, তুমি যদি সুখে থাকতে পারো তাহলে ওদের চুলকায় কেনো।",1,Code Switching,"আপু তোমাকে ধন্যবাদ এইভাবে সবাইকে জবাব দিয়ে শিক্ষা দেওয়ার জন্য,, তুমি যদি সুখে থাকতে পারো তাহলে ওদের চুলকায় কেনো।",NaN,24
3,gMeeozaMmIM,Laser Vision,2018-01-13T11:04:17Z,Surja Dighal Bari | Bangla Movie | Dolly Anwar | Zahirul Haque | Rowshan Jamil | Sheikh Niamat Ali,Entertainment,"এখানে আমার মনে হয় কেউ অভিনয় করে নাই। এটা একদম জীবন এর সাথে মিলে গেছে। একদম নিখুঁত অভিনয়। আগের দিন এর ছবি, নাটক দেখতে সত্যি অনেক সুন্দর লাগে অভিনয় যেন নয় যেন আসল শুধু ক্যামেরা বন্দী করা হয়ছে জীবনটাকে 🙂🙂।খুব সুন্দর লাগল ছবি টা 🙂🙂",1,Spelling,"এখানে আমার মনে হয় কেউ অভিনয় করে নাই। এটা একদম জীবন এর সাথে মিলে গেছে। একদম নিখুঁত অভিনয়। আগের দিন এর ছবি, নাটক দেখতে সত্যি অনেক সুন্দর লাগে অভিনয় যেন নয় যেন আসল শুধু ক্যামেরা বন্দী করা হয়েছে জীবনটাকে 🙂🙂।খুব সুন্দর লাগল ছবি টা 🙂🙂",NaN,55
4,Vhj-_avEjs0,Ritu Hossain,2023-06-26T12:00:08Z,রিতু নিজের টাকায় রুম ডেকোরেশন করলো | My Home Tour VLOG | Ritu Hossain | Rakib Hossain,Entertainment,ওওআপি আমি তোমার রোম দেখার জন্য অনেক অনেক অনেক হেপি😊,1,Spelling,আপু আমি তোমার কক্ষ দেখার জন্য অনেক অনেক অনেক খুশি😊,NaN,16


In [31]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['Video ID', 'Channel name ', 'Time of Publishing', 'Title', 'Genre', 'Comment', 'Error', 'Category', 'Correct Form', 'Unnamed: 9', 'input_token_len'],
    num_rows: 5022
})

In [32]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['Comment'], example['Correct Form']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs


In [33]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 24
attention_mask 24
labels 24
{'input_ids': [290, 406, 79, 14, 541, 38, 4576, 2082, 1068, 5, 6673, 2008, 17, 30, 881, 840, 11777, 756, 376, 520, 3157, 20, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [290, 406, 79, 14, 541, 38, 4576, 2082, 1068, 5, 158, 2630, 16332, 30, 881, 840, 11777, 756, 376, 520, 3157, 20, 2, 1]}


In [34]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=903e2275aa6aac4ccddeccb0f3aaa83668c2122a4466d1a7392940a371bc17ce
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [35]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [36]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [37]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.25.0
    Uninstalling accelerate-0.25.0:
      Successfully uninstalled accelerate-0.25.0


In [39]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=100,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")


In [40]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [ ]:
 trainer.train()

In [47]:
trainer.save_model('bangla_gec_model')

In [50]:


import locale
locale.getpreferredencoding = lambda: "UTF-8"



In [46]:
print('hello')

hello


In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'bangla_gec_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences,input_len):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=input_len, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=input_len,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text


In [52]:
references,predictions = [],[]
test_d = test_df[test_df['Error']==1]
test_d_sentence = test_d['Comment'].tolist()
test_d_len = test_d['input_token_len'].tolist()
test_d_ground = test_d['Correct Form'].tolist()
import nltk
nltk.download('punkt')
from nltk.util import ngrams
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
def calculate(reference,hypothesis):
    # Tokenize sentences into words
    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)

    # Create n-grams for reference and hypothesis
    reference_1grams = list(ngrams(reference_tokens, 1))
    hypothesis_1grams = list(ngrams(hypothesis_tokens, 1))
    reference_2grams = list(ngrams(reference_tokens, 2))
    hypothesis_2grams = list(ngrams(hypothesis_tokens, 2))

    # Calculate ROUGE scores
    rouge1_precision = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(reference_1grams)
    rouge1_recall = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(hypothesis_1grams)
    rouge2_precision = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(reference_2grams)
    rouge2_recall = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(hypothesis_2grams)

    # Calculate ROUGE-L using NLTK's sentence_bleu function
    smooth = SmoothingFunction().method4
    rougeL = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smooth)
    d = {
        "rouge1_precision":rouge1_precision,
        "rouge1_recall":rouge1_recall,
        "rouge2_precision":rouge2_precision,
        "rouge2_recall":rouge2_recall,
        "rouge_l":rougeL
    }
    return d

In [54]:
import nltk
from nltk.util import ngrams

# Function to calculate ROUGE-1, ROUGE-2, and ROUGE-L scores for a pair of texts
def calculate_rouge_scores(reference_tokens, system_tokens):
    def lcs(X, Y):
        m, n = len(X), len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i - 1] == Y[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

        return dp[m][n]

    # Calculate ROUGE-1 (unigram) scores
    reference_unigrams = set(reference_tokens)
    system_unigrams = set(system_tokens)
    overlap_rouge1 = len(reference_unigrams.intersection(system_unigrams))
    precision_rouge1 = overlap_rouge1 / len(system_unigrams)
    recall_rouge1 = overlap_rouge1 / len(reference_unigrams)
    r1_t = 1 if precision_rouge1 + recall_rouge1 == 0 else 0
    f1_rouge1 = 2 * (precision_rouge1 * recall_rouge1) / (precision_rouge1 + recall_rouge1 + r1_t)

    # Calculate ROUGE-2 (bigram) scores
    reference_bigrams = set(ngrams(reference_tokens, 2))
    system_bigrams = set(ngrams(system_tokens, 2))
    overlap_rouge2 = len(reference_bigrams.intersection(system_bigrams))
    precision_rouge2 = overlap_rouge2 / len(system_bigrams)
    recall_rouge2 = overlap_rouge2 / len(reference_bigrams)
    r2_t = 1 if precision_rouge2 + recall_rouge2 == 0 else 1
    f1_rouge2 = 2 * (precision_rouge2 * recall_rouge2) / (precision_rouge2 + recall_rouge2 + r2_t)

    # Calculate ROUGE-L scores
    lcs_length = lcs(reference_tokens, system_tokens)
    precision_rougeL = lcs_length / len(system_tokens)
    recall_rougeL = lcs_length / len(reference_tokens)
    rL_t = 1 if precision_rougeL + recall_rougeL == 0 else 0
    f1_rougeL = 2 * (precision_rougeL * recall_rougeL) / (precision_rougeL + recall_rougeL + rL_t)

    return {
        'ROUGE-1 Precision': precision_rouge1,
        'ROUGE-1 Recall': recall_rouge1,
        'ROUGE-1 F1': f1_rouge1,
        'ROUGE-2 Precision': precision_rouge2,
        'ROUGE-2 Recall': recall_rouge2,
        'ROUGE-2 F1': f1_rouge2,
        'ROUGE-L Precision': precision_rougeL,
        'ROUGE-L Recall': recall_rougeL,
        'ROUGE-L F1': f1_rougeL,
    }

# Function to calculate the average of ROUGE scores for an array of text pairs
def calculate_average_rouge_scores(reference_texts, system_texts):
    total_scores = {
        'ROUGE-1 Precision': 0,
        'ROUGE-1 Recall': 0,
        'ROUGE-1 F1': 0,
        'ROUGE-2 Precision': 0,
        'ROUGE-2 Recall': 0,
        'ROUGE-2 F1': 0,
        'ROUGE-L Precision': 0,
        'ROUGE-L Recall': 0,
        'ROUGE-L F1': 0,
    }

    num_pairs = len(reference_texts)

    for i in range(num_pairs):
        reference_text = reference_texts[i]
        system_text = system_texts[i]

        reference_tokens = nltk.word_tokenize(reference_text)
        system_tokens = nltk.word_tokenize(system_text)

        scores = calculate_rouge_scores(reference_tokens, system_tokens)

        for key, value in scores.items():
            total_scores[key] += value

    # Calculate the average scores
    average_scores = {key: value / num_pairs for key, value in total_scores.items()}
    
    return average_scores

# Example usage with an array of reference and system texts
reference_texts = test_d_ground
system_texts = [correct_grammar(test_d_sentence[i],num_return_sequences=2,input_len=test_d_len[i])[0] for i in range(len(test_d_sentence)) ]

average_scores = calculate_average_rouge_scores(reference_texts, system_texts)
print("Average ROUGE Scores:")
for key, value in average_scores.items():
    print(key + ": {:.4f}".format(value))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Average ROUGE Scores:
ROUGE-1 Precision: 0.8979
ROUGE-1 Recall: 0.8925
ROUGE-1 F1: 0.8947
ROUGE-2 Precision: 0.8182
ROUGE-2 Recall: 0.8126
ROUGE-2 F1: 0.5100
ROUGE-L Precision: 0.8993
ROUGE-L Recall: 0.8940
ROUGE-L F1: 0.8962


In [55]:
# random data

r = train_df['Comment'].tolist()[:20] + test_df['Comment'].tolist()[:20]
r_ln = train_df['input_token_len'].tolist()[:20] + test_df['input_token_len'].tolist()[:20]
reference_texts = r
for i in range(len(r)):
    print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')


[input:   ] সেলুট আপনাকে সত্যি কথা তুলে ধরার জন্য আল্লাহ তায়ালা আপনাকে নেক হায়াত দান করুক আমিন and [output:   ] সেলুট আপনাকে সত্যি কথা তুলে ধরার জন্য আল্লাহ তায়ালা আপনাকে নেক হায়াত দান করুক আমিন
[input:   ] তোমারে পেয়ে গেলে হয়তো এত সুন্দর গানটা বার বার শুনা হয়তো না😢 and [output:   ] তোমারে পেয়ে গেলে হয়ত এত সুন্দর গানটা বার বার শুনা হয়তো না😢
[input:   ] আমার প্রতিদিনের রুটিনে এই গানটি থাকে তবে ডাউনলোড কখনো করিনি সবসময় সার্চ করে দেখি 😊 and [output:   ] আমার প্রতিদিনের রুটিনে এই গানটি থাকে তবে ডাউনলোড কখনো করিনি সবসময় সার্চ করে দেখি 😊
[input:   ] সকল মুসলিম দেশ গুলো ফিলিস্তিনের পাশে থাকা ঈমানি দায়িত্ব and [output:   ] সকল মুসলিম দেশ গুলো ফিলিস্তিনের পাশে থাকা ইমানি দায়িত্ব
[input:   ] কে কে নবীজি কে ভালো বাসেন? 🥀🥀🥀 and [output:   ] কে কে নবীজি কে ভালোবাসেন? 
[input:   ] এই নাটকটা দেখার পর আবারও স্বরণ হয়ে গেলো যে, তোমাকে না পাওয়ার ব্যথা টা খুবই তীব্র ছিল প্রিয়... and [output:   ] এই নাটকটা দেখার পর আবারও স্মরণ হয়ে গেলো যে, তোমাকে না পাওয়ার ব্যথা টা খুবই তীব্র ছিল প্রিয়.

In [56]:
# error data

import time

r = (test_df[test_df['Error']==1])['Comment'].tolist()
r_ln = (test_df[test_df['Error']==1])['input_token_len'].tolist()
reference_texts = r
start_time = time.time()
for i in range(len(r)):
    x = r[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
#     print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')
end_time = time.time()
num_iterations = len(r)
average_inference_time = (end_time - start_time) / num_iterations
print(f"Total Inference Time: {end_time-start_time}")
print(f"Average Inference Time: {average_inference_time:.4f} seconds")

Total Inference Time: 1093.1689569950104
Average Inference Time: 0.3515 seconds


In [63]:
r_gn = (test_df[test_df['Error']==1])['Correct Form'].tolist()

In [64]:
predicted_score,predicted_sentences = [],[]
for i in range(len(r_gn)):
    x = r_gn[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
    predicted_sentences.append(y)
    rouge_score_v = calculate_rouge_scores(x, y)
    predicted_score.append(rouge_score_v['ROUGE-L F1'])
# test_df['ROUGE-L F1'] = predicted_sentence



In [65]:
e_df = test_df[test_df['Error']==1]
e_df['ROUGE-L F1'] = predicted_score
e_df['Predicted Form'] = predicted_sentences


/tmp/ipykernel_26/2934693448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['ROUGE-L F1'] = predicted_score
/tmp/ipykernel_26/2934693448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['Predicted Form'] = predicted_sentences


In [66]:
e_df.sort_values(by='ROUGE-L F1', inplace=True,ascending=False)

/tmp/ipykernel_26/3169983084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df.sort_values(by='ROUGE-L F1', inplace=True,ascending=False)


In [70]:
e_df.head(20)[1:].T

,1578,1594,1593,1590,1587,549,550,551,1584,1581,1579,1577,1595,557,1573,1571,1569,1566,562
Video ID,y80X-8VD8fU,MfdIW-gFk4w,2z8Nhs0Qq4E,AfTVndf8yO0,cqpQFGnT8X8,gMeeozaMmIM,s21NZ19thvw,aLMmkiy6dNA,aLMmkiy6dNA,lD9IEhtE7sI,u3L-6T8pMaY,dvbhI7lB5Yg,EEJtX-nVXM8,kITxvmWBFek,oPysZIjBur4,Np5gpPNhuqI,MGXEzkBmtA0,acagRREpim0,xSgrBJn5q68
Channel name,Banglavision DRAMA,ATN News,Open T School,ATN News,Saif Zohan,Laser Vision,Holy FM,T Sports,T Sports,SOMOY TV,Channel 24,বিস্ময় বাংলা,CMV,Film Park BD,BBC News বাংলা,CD PLUS Entertainment,SOMOY TV,Mama Problem?,Rtv News
Time of Publishing,2023-07-13T13:31:37Z,2023-10-04T13:10:11Z,2022-12-14T13:05:13Z,2023-07-18T12:00:08Z,2022-06-30T10:00:08Z,2018-01-13T11:04:17Z,2020-12-16T13:38:50Z,2021-12-22T14:43:56Z,2021-12-22T14:43:56Z,2022-01-03T18:56:05Z,2022-11-10T14:15:07Z,2023-05-01T11:00:23Z,2023-03-19T09:01:03Z,2018-12-12T04:03:43Z,2018-03-06T06:38:04Z,2020-12-08T14:20:37Z,2023-06-16T16:20:00Z,2021-07-11T16:48:42Z,2023-07-21T10:46:47Z
Title,Putuler Songshar | পুতুলের সংসার | Tanjin Tisha | Sohel Mondal | Rinku | Bangla New Natok 2023,বাংলাদেশ ক্রিকেটে নাটকের শেষ কোথায়? | Tamim Iqbal | Bangladesh Cricket | BCB | ATN News,কে ছিলেন জিয়াউর রহমান | Who was Ziaur Rahman | Biography | Information |,হিরো আলম - শামীম ওসমান ইস্যুতে যে বার্তা দিল যুক্তরাষ্ট্র | Hero Alam | Shamim Osman | USA |ATN News,Amar Mon Mojaiya Re X Doyal | আমার মন মজাইয়ারে | Slowed & Reverb | Saif Zohan Bangla New Song 2022,Surja Dighal Bari | Bangla Movie | Dolly Anwar | Zahirul Haque | Rowshan Jamil | Sheikh Niamat Ali,কালজয়ী দেশাত্মবোধক গান | Eta Bangladesh | Muhib Khan | Holy FM,Highlights | চ্যাম্পিয়ন বাংলাদেশ! | Bangladesh Vs India | Saff U-19 Women's Championship | T Sports,Highlights | চ্যাম্পিয়ন বাংলাদেশ! | Bangladesh Vs India | Saff U-19 Women's Championship | T Sports,স্পা সেন্টারের আড়ালে অনৈতিক কার্যকলাপ | Spa Center | Dhaka News | Somoy TV,ঝাল মুড়ির নাম ‘চোখের পানি’ কেন? | মুখরোচক | Mukhorochok | New Market Chokher Pani | Channel 24,যে ৫টি কারনে ২০২৩ বিশ্বকাপ জিততে পারে বাংলাদেশ || 5 Reason Bangladesh Could Win World Cup 2023,Bou Bojhena | বউ বোঝেনা | Bangla Natok | Tawsif Mahbub | Keya Payel | Eid Natok | Bangla Natok 2023,Mukti Ekattur | Shortfilm Based on Libration war of Bangladesh against Pakistan | 1971 | FilmPark,বাংলাদেশে প্রথমবারের মত মাথা জোড়া লাগা জমজ শিশুদের পৃথক করতে অস্ত্রোপচার। BBC Bangla,Best Of Moushumi | Bangla Movie Songs | Vol 1 | 5 Superhit Movie Video Songs,কোন পথে দেশের রাজনীতি? | Bangladesh Politics | PM Sheikh Hasina | Begum Khaleda Zia | Somoy TV,আর্জেন্টিনা VS ব্রাজিল|ফাইনাল|কোপা আমেরিকা ২০২১|Football Bangla Funny Dubbing Mama Problem New,চাপাতি দিয়ে কুপিয়ে যুবলীগ কর্মীকে হ'ত্যা | Jubo League workers | Murder | Rtv News
Genre,Entertainment,Sports,Miscellaneous,Politics,Entertainment,Entertainment,Entertainment,Sports,Sports,News,Miscellanous,Miscellaneous,Entertainment,Entertainment,News,Entertainment,News,Entertainment,News
Comment,নাটক টি সত্যি মন ছুয়ে গেছে❤😢😢😢😢,আমি জোরালো আবেদন জানাচ্ছি তামিম কে দলে ফেরানোর জন্য আরো আবেদন জান্নাচ্ছি যারা তামিম কে বাদ দিয়েছে তাদের যেনো আইনের আহতায় আনা হয়,বাংলাদেশের জন্য তিনি এতো কিছু করলো আর এর বিনিময়ে তার পরিবারের আজকে এই পরিনতি। আস্তাগফিরুল্লাহ,হিরো আলম ভাই হার না মানা এক জাতি আপনি চালিয়ে জান ইনশাআল্লাহ আপনার এক দিন বিজয় হবে,নিজের কাছে আগে এই গান গুলো সবচেয়ে বেশি খারাপ লাগতো আর এখন এই গান গুলো সব চেয়ে বেশি প্রিয় হইয়া গেছে😅😊,মানুষের বাস্তব জীবন থেকে নেওয়া সত্যি ঘটনা যা আমরা শুধুই সিনেমা হিসেবে যা দেখে থাকি,"আল্লাহু আকবার, আল্লাহু আকবার প্রতিধ্বনি পৌঁছে যাক আল্লাহপাকের আরশে আজিমে এবং বিজয় শুরু হোক পৃথিবীর শ্রেষ্ঠ ধর্ম ইসলামের Amin",পুরো টুর্নামেন্টে ১টা গোলও হজম না করে মোট ২০টা গোল করে চ্যাম্পিয়ন হওয়া বাংলার তরুণ বাঘিনীদের জন্য অভিন্দন ও ভালোবাসা,ভাগ্য খুব খারাপ না হলে অনেকগুলো গোল এর মুখ দেখতে পারতাম অনেক ভালো খেলছে আজকে পুরো টিম অনেক অনেক ধন্যবাদ এবং শুভকামনা,"এই ধরনের অভিযান আরো বেশিকরে করতে হবে এবং শাস্তির পরিমানটা বেশি করতে হবে,ধন্যবাদ সাংবাদিক ও পুলিশ বাহিনিকে",ভাইরে যেটা খামু সেখানে ভেজাল,ইনশাআল্লাহ মুশফিক ভাই ওয়াল্ডকাপ হাতে নিবে কমেন

In [69]:
e_df.tail(20).T

,181,1454,2973,789,1009,1231,1602,1352,423,516,1200,2977,2354,1605,2336,1397,1121,1979,3032,872
Video ID,HFRHEPyouA8,JOisal71jns,-m8yrtD3Ak8,Lg7NmCH8f_k,Cs56lXclISk,oZjyusPtzkY,5DL10pkgnOw,DpN5gVjZVm8,DwgIzrQrnnE,Y5j9KLtRb2g,J4UDBAueqXA,A9al-oYmKnY,7tmPUNYRdt8,q36_ZA50-iM,BpRmJAWr0e8,0zEizwPZUzw,u88ycLVx5ho,RZDSt0gGmt0,hRIooJ7Pkb8,gMeeozaMmIM
Channel name,Info Hunter,Dhruba Clips,Farjana Drawing Academy,Chorki,Maasranga News,Jamuna TV,Enayet Chowdhury,Team bachelor official,Family Entertainment Bd,BBC News বাংলা,Somoy Entertainment,Mahakash TV,Channel 24,T Sports,বিস্ময় বাংলা,Ring Media,Maasranga News,Somoy Entertainment,Life_2711,Laser Vision
Time of Publishing,2022-11-16T13:00:17Z,2022-08-15T08:00:12Z,2017-03-13T14:36:50Z,2022-04-02T13:00:12Z,2022-08-30T08:23:39Z,2022-10-31T14:27:28Z,2023-09-28T11:00:04Z,2023-09-06T14:30:07Z,2023-08-06T08:00:07Z,2021-08-19T06:00:11Z,2023-05-27T13:00:09Z,2022-08-17T04:17:32Z,2023-07-29T13:30:03Z,2022-01-28T15:43:23Z,2022-02-04T11:00:07Z,2020-05-30T14:30:09Z,2023-12-17T14:37:44Z,2023-03-24T16:05:36Z,2022-05-13T15:00:30Z,2018-01-13T11:04:17Z
Title,লালবাগে সিদ্দিক ভাইয়ের কারেলি গোশতের স্পেশাল নেহারি | Info Hunter,বিয়েতে খাওয়ার পর দই না পেয়ে বর পক্ষের সাথে কনে পক্ষের তুমুল মারামারি,How to draw scenery of Language Movement Step by step || Art video,Pett Kata Shaw - ষ | Official Trailer | Chorki Original Series | Nuhash | Chanchal | Pritom | Shohel,খালেদা জিয়া’র ব্যাপারে আর কত করবো ? প্রশ্ন প্রধানমন্ত্রীর | Sheikh Hasina | Khaleda Zia,ঘোড়া কীভাবে সাপের কামড় থেকে মানুষকে বাঁচায়? | Why horse is used for antivenom? | Jamuna TV,হিন্দি সিনেমা আনা বন্ধ হলেই সব ঠিক হবে না যে কারণে । Bollywood in Bangladesh | Enayet Chowdhury,নির্বাচনের আগে হিরো আলম আমার সাথে দেখা করেছিল | ব্যারিস্টার আন্দালিব রহমান পার্থ,এসএসসি পরীক্ষার মিষ্টি | SSC Exam Result | Bangla Funny Video | Family Entertainment bd | Desi Cid,তালেবান যোদ্ধারা আফগানিস্তান ও পাকিস্তান সীমান্তের পাহারায় | BBC Bangla,শাকিব খান আমার নায়ক হবার যোগ্যই না: শান্তি | Shanti Rehman Nazia | Content Creator | Shakib Khan,নাসার বিজ্ঞানীরা চাঁদের মাটিতে গাছ লাগিয়ে সবাইকে অবাক করে দিলো | Nasa Started Farming on the Moon,বিএনপির অবস্থান কর্মসূচি ঘিরে রণক্ষেত্র ঢাকার বিভিন্ন প্রবেশমুখ | BNP & Police | Channel 24,৬৬ বলে ১৭৭ রান করে টি টোয়েন্টিতে গেইলকেও ছাড়িয়ে গেলেন আমান। | ACT | T Sports,মাহমুদুল্লাহর সেরা ১০টি ইনিংস || 10 Greatest Innings of Mahmudullah Riyad || Bissoy Bangla,Fake ID | Afran Nisho | Tasnia Farin | Noyeem Imtiaz Neamul | New Eid Natok 2020,"আওয়ামী লীগের কারা বাদ পড়লেন, জাতীয় পার্টির কারা পেলেন সেসব আসন | Maasranga News",নিজেকে ইনফ্লুয়েন্সার নয় কন্টেন্ট ক্রিয়েটর ভাবেন সুনেহরা | Sunehra Tasnim | Content Creator |Somoy TV,মনে পড়ে? ৮০ দশকের বিটিভির সেই সুর। 80's morning show music of Bangladesh Television (BTV)। Life_2711,Surja Dighal Bari | Bangla Movie | Dolly Anwar | Zahirul Haque | Rowshan Jamil | Sheikh Niamat Ali
Genre,Miscellanous,Miscellaneous,Miscellaneous,Entertainment,Politics,Miscellaneous,Miscellaneous,Politics,Entertainment,News,Entertainment,Miscellaneous,Politics,Sports,Sports,Miscellaneous,Politics,Entertainment,Entertainment,Entertainment
Comment,টেষ্ট করে দেখতে হবে দেখছি,ভাই মোভি টার নাম কি,এত তারা তারি আকছেন কেন,প্রথম কোনো বাংলা হরর ফিল্মের ট্রেইলার দেখতেই ভয় পেলাম। অনেক এক্সাইটেড 🥰,আনটি আপনার আবার কি পাওয়ার পাওয়ার তো ওই উপর আল্লাহর,দুঃখ বিসয় এখণো ভারত নির্ভর সিল,ভাইয়ের ভিডিওর এডিটরকে মন থেকে লাল সালাম 🫡🫡🫡 জোস এডিট😮😮,একজন জোকারি আরেক জোকারের মর্ম বোঝে,রেজাল্টের আগেই এটি রিলিজ করা উচিত ছিল।😑,"আমি মনে করি তালেবান বিধি নিষেধের ব্যবস্থা খুবই ভালো, যেখানে ধর্ষণ দূর্নিতি অপরাধ কমে যাবে,যেখানে মানুষ ভালো কিছু শিক্ষা দিবে,প্রতিটি রাস্ট্র প্রধান হিয় একজন অভিবাবক,যেমন আমার বাবার ২ ভাই আমরা একজন যদি সমাজ ভালো কাজ গুলো করে থাকে,আর অন্য জন খারাপ কাজ করে তখন কি আমার বাবার দায়িত্বনা যে থাকে শাসন করা? যেনো আমি মাদক আশক্ত না হয়ে যাই,যেনো আমি সন্ত্রাসী না হয়ে যাই,যেনো আমি নারী কে অসন্মান না করতে পারি,একজন নারী যখন চেহেরাটা খোলা থাকে তখন একটা ছেলে যদি ওই নারী দিখে থাকি খারাপ কিছু বলে এটা নারী অসন্মান, তা হলে আমার বাবা আমাক্র শাসন করবে আমার 